In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import json


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\ProgramData\anaconda3\envs\gym-equipment-detection\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\ProgramData\anaconda3\envs\gym-equipment-detection\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\anaconda3\envs\gym-equipment-detection\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\anaconda3\envs\gym-equipment-detection\lib\site-packages\traitlets

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
def extract_frames(video_path, output_folder, frame_rate=5):
    """
    Extract frames from a video and save them as images.
    
    Args:
        video_path (str): Path to the input video file.
        output_folder (str): Directory to save the extracted frames.
        frame_rate (int): Save one frame every 'frame_rate' frames.
    """
    # Ensure output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    cap = cv2.VideoCapture(video_path)
    count = 0
    frame_id = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Save every 'frame_rate' frames
        if frame_id % frame_rate == 0:
            frame_filename = os.path.join(output_folder, f"frame_{count}.jpg")
            cv2.imwrite(frame_filename, frame)
            print(f"Saved {frame_filename}")
            count += 1

        frame_id += 1

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
video_folder = 'videos/'
frames_output_folder = 'frames/'

# Ensure the frames output directory exists
if not os.path.exists(frames_output_folder):
    os.makedirs(frames_output_folder)

# Process each video file
for video_file in os.listdir(video_folder):
    if video_file.endswith(('.mp4', '.avi', '.mov')):
        video_path = os.path.join(video_folder, video_file)
        video_name = os.path.splitext(video_file)[0]
        video_frames_folder = os.path.join(frames_output_folder, video_name)
        extract_frames(video_path, video_frames_folder, frame_rate=5)  # Adjust frame_rate as needed


In [ ]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize the Mediapipe Pose model
pose = mp_pose.Pose(
    static_image_mode=True,
    min_detection_confidence=0.5,
    model_complexity=1  # You can set this to 0 for faster processing
)


In [ ]:
def process_image(image_path):
    """
    Process an image to detect pose landmarks using Mediapipe.

    Args:
        image_path (str): Path to the input image.

    Returns:
        keypoints (list): List of detected keypoints.
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image {image_path}")
        return None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        keypoints = []
        h, w, c = image.shape
        for lm in results.pose_landmarks.landmark:
            # Convert normalized coordinates to pixel values
            x = int(lm.x * w)
            y = int(lm.y * h)
            z = lm.z  # z is relative to depth, in meters
            visibility = lm.visibility
            keypoints.append({
                'x': x,
                'y': y,
                'z': z,
                'visibility': visibility
            })
        if visualize:
            visualize_keypoints(image_path, results)
        return keypoints
    else:
        print(f"No pose landmarks detected in {image_path}")
        return None


In [ ]:

def visualize_keypoints(image_path, results):
    """
    Draw pose landmarks on the image and display it.

    Args:
        image_path (str): Path to the input image.
        results: Mediapipe results object containing pose landmarks.
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to read image {image_path}")
        return

    # Draw pose landmarks on the image.
    annotated_image = image.copy()
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
    )

    # Convert BGR image to RGB for displaying
    annotated_image_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)

    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 10))
    plt.imshow(annotated_image_rgb)
    plt.axis('off')
    plt.show()


In [ ]:
# Load the ResNet50 model pre-trained on ImageNet, excluding the top layers
base_model = ResNet50(weights='imagenet',
                      include_top=False,
                      input_tensor=Input(shape=(224, 224, 3)))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display the model architecture
model.summary()


In [ ]:
# Define training parameters
epochs = 3

# Train the model
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=test_ds
)


In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_ds, verbose=1)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(14,5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], 'b-', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], 'r--', label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], 'b-', label='Training Loss')
plt.plot(history.history['val_loss'], 'r--', label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()
